# Table of Contents
 <p><div class="lev1"><a href="#Summary"><span class="toc-item-num">1&nbsp;&nbsp;</span>Summary</a></div><div class="lev1"><a href="#Imports"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></div><div class="lev1"><a href="#Load-data"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load data</a></div><div class="lev1"><a href="#Map-to-UniProt"><span class="toc-item-num">4&nbsp;&nbsp;</span>Map to UniProt</a></div><div class="lev2"><a href="#Construct-RefSeq-/-UniProt-maps"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Construct RefSeq / UniProt maps</a></div><div class="lev2"><a href="#Apply-RefSeq-/-UniProt-maps"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Apply RefSeq / UniProt maps</a></div><div class="lev2"><a href="#Convert-to-UniProt-mutations"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Convert to UniProt mutations</a></div><div class="lev3"><a href="#mmc3_uniprot_df"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>mmc3_uniprot_df</a></div><div class="lev3"><a href="#mmc3_uniprot_df_good"><span class="toc-item-num">4.3.2&nbsp;&nbsp;</span>mmc3_uniprot_df_good</a></div><div class="lev3"><a href="#Merge-back"><span class="toc-item-num">4.3.3&nbsp;&nbsp;</span>Merge back</a></div><div class="lev1"><a href="#Combine-datasets"><span class="toc-item-num">5&nbsp;&nbsp;</span>Combine datasets</a></div><div class="lev3"><a href="#uniprot_interface_mutation_df"><span class="toc-item-num">5.0.1&nbsp;&nbsp;</span>uniprot_interface_mutation_df</a></div>

# Summary

Process the Taipale dataset.

Link to paper: http://www.sciencedirect.com/science/article/pii/S0092867415004304

----

# Imports

In [3]:
%run imports.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-07-15 15:38:41.123936


In [4]:
NOTEBOOK_NAME = 'taipale'
os.makedirs(NOTEBOOK_NAME, exist_ok=True)
os.environ['NOTEBOOK_NAME'] = NOTEBOOK_NAME

os.environ['DB_PORT'] = '8308'

In [6]:
%run mysqld.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-07-15 15:41:39.896144


In [7]:
# Start a database connection
db_remote = datapkg.MySQL(
    os.environ['DATAPKG_CONNECTION_STR'] + '/staging', 
    shared_folder=os.environ['NOTEBOOK_NAME'],
    storage_host=None,
    echo=False,
    db_engine='MyISAM'
)


# Load data

Table S3. Protein-Protein Interaction Profiles and Orthogonal GPCA Validation, Related to Figure 4.  

**Table 1**:

> Protein-protein interaction (PPI) scores, listing all interactions tested in Y2H and mutation-specific perturbations. In the “Allele_ID” column, “_0” represents wild-type proteins. For “Y2H_score,” “1” denotes positive interactions, while “0” denotes no or negative interactions. “Interactor_Gene_ID” denotes the Entrez ID for the interaction partner gene. “in Rolland et al” indicates if a given PPI was tested (denoted by “1”) in the dataset (Rolland et al., 2014) or not (denoted by “0”).


**Table 2**:

> In vivo GPCA scores for PPI validation.


**Table 3**:

> Edgotype classification for all the mutations/variants based on their PPI profiles. Mutations/alleles with 1 or more interaction partners in the corresponding wild-type proteins are all included.


In [5]:
# Load tables
mmc3_table1_df = local.load_taipale_mmc3('Table S3A')
mmc3_table2_df = local.load_taipale_mmc3('Table S3B')
mmc3_table3_df = local.load_taipale_mmc3('Table S3C')

Table S3A : (2261, 10)
Table S3B : (858, 12)
Table S3C : (527, 7)


In [6]:
display(mmc3_table1_df.head(2))
print(mmc3_table1_df.shape[0])

,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,Interactor_symbol,Interactor_Gene_ID,Y2H_score,In_Rolland_et_al,refseq_base_id,refseq_mutation,refseq_mutation_pos
0,Wild-type,AANAT,15,15_0,NaN,NaN,BHLHE40,8553,1,1,NaN,NaN,NaN
1,Wild-type,AANAT,15,15_0,NaN,NaN,MDFI,4188,1,1,NaN,NaN,NaN


2261


In [7]:
display(mmc3_table2_df.head(2))
print(mmc3_table2_df.shape[0])

,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,Interactor_symbol,Interactor_Gene_ID,Interaction_category,GPCA_score1,GPCA_score2,GPCA_average_score,refseq_base_id,refseq_mutation,refseq_mutation_pos
0,Wild-type,AANAT,15,15_0,NaN,NaN,MDFI,4188,Y2H_positive,4.040826,4.698746,4.369786,NaN,NaN,NaN
1,Wild-type,AANAT,15,15_0,NaN,NaN,BHLHE40,8553,Y2H_positive,-1.458909,-1.131614,-1.295262,NaN,NaN,NaN


858


In [8]:
display(mmc3_table3_df.head(2))
print(mmc3_table3_df.shape[0])

,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,Edgotype_class,refseq_base_id,refseq_mutation,refseq_mutation_pos
0,Disease mutation,AANAT,15,15_22963,NM_001088:c.385G>A,NP_001079:p.A129T,Edgetic,NP_001079,A129T,129
1,Non-disease variant,AANAT,15,15_900207,NM_001088:c.8C>T,NP_001079:p.T3M,Quasi-wild-type,NP_001079,T3M,3


527


# Map to UniProt

## Construct RefSeq / UniProt maps

In [9]:
# Create disctionaries for converting gene symbol to refseq / uniprot id
# (`gene_id_to_uniprot` only works for the partner UniProt ID!)
gene_symbol_to_refseq_1, gene_id_to_uniprot_1 = local.get_conversion_tables(mmc3_table1_df)
gene_symbol_to_refseq_2, gene_id_to_uniprot_2 = local.get_conversion_tables(mmc3_table2_df)

gene_symbol_to_refseq = pd.concat(
    [gene_symbol_to_refseq_1, gene_symbol_to_refseq_2],
    ignore_index=True).drop_duplicates()
gene_id_to_uniprot = pd.concat(
    [gene_id_to_uniprot_1, gene_id_to_uniprot_2],
    ignore_index=True).drop_duplicates()

display(gene_symbol_to_refseq.head())
print(gene_symbol_to_refseq.shape[0])

display(gene_id_to_uniprot.head())
print(gene_id_to_uniprot.shape[0])

,Symbol,refseq_base
0,AANAT,NP_001079
1,ACTB,NP_001092
2,ACTG1,NP_001605
3,ACTN2,NP_001094
4,ACTN4,NP_004915


223


,identifier_id,uniprot_id
0,10016,O75340
1,10016,Q53FC3
2,100288797,Q8WW34
3,10062,B4DXU5
4,10062,F1D8N1


1376


## Apply RefSeq / UniProt maps

In [10]:
# Covert gene symbol to refseq id (does not work for some partner proteins where we have never seen the refseq id.)
display(HTML('<h3>Table 1:'))
mmc3_table1_df2 = local.convert_interacting_partner(mmc3_table1_df, gene_symbol_to_refseq, gene_id_to_uniprot)
display(HTML('<h3>Table 2:'))
mmc3_table2_df2 = local.convert_interacting_partner(mmc3_table2_df, gene_symbol_to_refseq, gene_id_to_uniprot)

All rows:                                                   (3299, 15)
Unique interfaces / mutations:                              (1633, 3) <--
Unique interfaces / mutations mapped to uniprot:            (1633, 3)
Unique interfaces / mutations mapped to uniprot and refseq: (1633, 3)


All rows:                                                   (3284, 15)
Unique interfaces / mutations:                              (1633, 3) <--
Unique interfaces / mutations mapped to uniprot:            (1633, 3)
Unique interfaces / mutations mapped to uniprot and refseq: (1633, 3)


,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,Interactor_symbol,Interactor_Gene_ID,Y2H_score,In_Rolland_et_al,refseq_base_id,refseq_mutation,refseq_mutation_pos,refseq_base_id_1,uniprot_id_2
0,Wild-type,AANAT,15,15_0,NaN,NaN,BHLHE40,8553,1,1,NaN,NaN,NaN,NP_001079,O14503
1,Wild-type,AANAT,15,15_0,NaN,NaN,BHLHE40,8553,1,1,NaN,NaN,NaN,NP_001079,Q6IB83
2,Wild-type,AANAT,15,15_0,NaN,NaN,MDFI,4188,1,1,NaN,NaN,NaN,NP_001079,B1AKB6
3,Wild-type,AANAT,15,15_0,NaN,NaN,MDFI,4188,1,1,NaN,NaN,NaN,NP_001079,Q99750


All rows:                                                   (1323, 17)
Unique interfaces / mutations:                              (492, 3) <--
Unique interfaces / mutations mapped to uniprot:            (492, 3)
Unique interfaces / mutations mapped to uniprot and refseq: (492, 3)


All rows:                                                   (1323, 17)
Unique interfaces / mutations:                              (492, 3) <--
Unique interfaces / mutations mapped to uniprot:            (492, 3)
Unique interfaces / mutations mapped to uniprot and refseq: (492, 3)


,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,Interactor_symbol,Interactor_Gene_ID,Interaction_category,GPCA_score1,GPCA_score2,GPCA_average_score,refseq_base_id,refseq_mutation,refseq_mutation_pos,refseq_base_id_1,uniprot_id_2
0,Wild-type,AANAT,15,15_0,NaN,NaN,MDFI,4188,Y2H_positive,4.040826,4.698746,4.369786,NaN,NaN,NaN,NP_001079,B1AKB6
1,Wild-type,AANAT,15,15_0,NaN,NaN,MDFI,4188,Y2H_positive,4.040826,4.698746,4.369786,NaN,NaN,NaN,NP_001079,Q99750
2,Wild-type,AANAT,15,15_0,NaN,NaN,BHLHE40,8553,Y2H_positive,-1.458909,-1.131614,-1.295262,NaN,NaN,NaN,NP_001079,O14503
3,Wild-type,AANAT,15,15_0,NaN,NaN,BHLHE40,8553,Y2H_positive,-1.458909,-1.131614,-1.295262,NaN,NaN,NaN,NP_001079,Q6IB83


In [11]:
mmc3_table1_df2.drop_duplicates(['refseq_base_id', 'refseq_mutation']).shape

(528, 15)

## Convert to UniProt mutations

### mmc3_uniprot_df

In [12]:
# Create a temporary table in the database (`staging`.`taipale_mmc3`)
data_to_upload = (
    pd.concat(
        [mmc3_table1_df2[['refseq_base_id', 'refseq_mutation', 'refseq_mutation_pos']],
         mmc3_table2_df2[['refseq_base_id', 'refseq_mutation', 'refseq_mutation_pos']]],
        ignore_index=True
    )
    .dropna()
    .drop_duplicates()
)

In [13]:
# Print stats
print2(
    "Mutations from 'mmc3_table1_df2':", 
    mmc3_table1_df2[['refseq_base_id', 'refseq_mutation', 'refseq_mutation_pos']].drop_duplicates().shape)
print2(
    "Mutations from 'mmc3_table2_df2':",
    mmc3_table2_df2[['refseq_base_id', 'refseq_mutation', 'refseq_mutation_pos']].drop_duplicates().shape)
print2(
    "Combined:", 
    data_to_upload.shape)

Mutations from 'mmc3_table1_df2':                           (528, 3)
Mutations from 'mmc3_table2_df2':                           (111, 3)
Combined:                                                   (527, 3)


In [14]:
data_to_upload.to_sql('taipale_mmc3', db_remote.engine, schema='staging', if_exists='replace')

In [15]:
# Convert RefSeq ID and position to UniProt (canonical) ID and position
sql_query = """
SELECT
-- RefSeq
m.index,
m.refseq_base_id,
m.refseq_mutation,
m.refseq_mutation_pos,
ux.active refseq_active,
ucuc.uniparc_sequence refseq_sequence,

-- UniProt
uxc.id uniprot_id,
FIND_IN_SET(refseq_mutation_pos, a2b) uniprot_mutation_pos,
uxc.active uniprot_active,
# Don't use UniParc sequence because it might be different version than ELASPIC
us.uniprot_sequence uniprot_sequence

FROM staging.taipale_mmc3 m
JOIN uniparc_human.uniparc_xref ux ON (ux.type = 'RefSeq' AND ux.id = m.refseq_base_id)
JOIN uniparc_human.uniparc2uniparc_canonical_mapping ucuc USING (uniparc_id)
JOIN uniparc_human.uniparc_xref uxc ON (uxc.uniparc_id = ucuc.uniparc_canonical_id AND \
                                        uxc.type = 'UniProtKB/Swiss-Prot')
JOIN uniprot_kb.uniprot_sequence us ON (us.uniprot_id = uxc.id);
"""
mmc3_uniprot_df = pd.read_sql_query(sql_query, db_remote.engine)

In [16]:
mmc3_uniprot_df.to_pickle(op.join(NOTEBOOK_NAME, 'mmc3_uniprot_df.pkl'))

In [17]:
mmc3_uniprot_df_bak = mmc3_uniprot_df.copy()

### mmc3_uniprot_df_good

In [18]:
mmc3_uniprot_df = mmc3_uniprot_df_bak.copy()

In [19]:
display(mmc3_uniprot_df.head())
print(mmc3_uniprot_df.shape)

,index,refseq_base_id,refseq_mutation,refseq_mutation_pos,refseq_active,refseq_sequence,uniprot_id,uniprot_mutation_pos,uniprot_active,uniprot_sequence
0,4,NP_001079,A129T,129.0,N,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,Q16613,129,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...
1,4,NP_001079,A129T,129.0,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,Q16613,129,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...
2,8,NP_001079,T3M,3.0,N,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,Q16613,3,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...
3,8,NP_001079,T3M,3.0,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,Q16613,3,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...
4,19,NP_001092,R183W,183.0,N,MDDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVG...,P60709,183,Y,MDDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVG...


(1008, 10)


In [20]:
mmc3_uniprot_df['uniprot_mutation'] = (
    mmc3_uniprot_df[['refseq_mutation', 'uniprot_mutation_pos']]
    .apply(lambda x: x[0][0] + str(x[1]) + x[0][-1], axis=1)
)

In [21]:
mmc3_uniprot_df['refseq_mutation_matches_sequence'] = (
    mmc3_uniprot_df[['refseq_mutation', 'refseq_sequence']]
    .apply(lambda x: ascommon.sequence_tools.mutation_matches_sequence(*x), axis=1)
)

In [22]:
mmc3_uniprot_df['uniprot_mutation_matches_sequence'] = (
    mmc3_uniprot_df[['uniprot_mutation', 'uniprot_sequence']]
    .apply(lambda x: ascommon.sequence_tools.mutation_matches_sequence(*x), axis=1)
)

In [23]:
display(mmc3_uniprot_df.head())
print(mmc3_uniprot_df.shape)

,index,refseq_base_id,refseq_mutation,refseq_mutation_pos,refseq_active,refseq_sequence,uniprot_id,uniprot_mutation_pos,uniprot_active,uniprot_sequence,uniprot_mutation,refseq_mutation_matches_sequence,uniprot_mutation_matches_sequence
0,4,NP_001079,A129T,129.0,N,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,Q16613,129,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,A129T,True,True
1,4,NP_001079,A129T,129.0,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,Q16613,129,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,A129T,True,True
2,8,NP_001079,T3M,3.0,N,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,Q16613,3,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,T3M,True,True
3,8,NP_001079,T3M,3.0,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,Q16613,3,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,T3M,True,True
4,19,NP_001092,R183W,183.0,N,MDDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVG...,P60709,183,Y,MDDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVG...,R183W,True,True


(1008, 13)


In [24]:
# All
df = mmc3_uniprot_df
print2("Not 'uniprot_mutation_pos':", (df['uniprot_mutation_pos'] == 0).sum())
print2("Not 'uniprot_mutation_matches_sequence':", (~df['refseq_mutation_matches_sequence']).sum())
print2("Not 'refseq_mutation_matches_sequence':", (~df['refseq_mutation_matches_sequence']).sum())
print2("Not 'uniprot_active':", (df['uniprot_active'] == 'N').sum())
print2("Not 'refseq_active':", (df['refseq_active'] == 'N').sum())
print(df.shape[0], df['index'].drop_duplicates().shape[0])

Not 'uniprot_mutation_pos':                                 10
Not 'uniprot_mutation_matches_sequence':                    15
Not 'refseq_mutation_matches_sequence':                     15
Not 'uniprot_active':                                       40
Not 'refseq_active':                                        459
1008 526


In [25]:
# Select the best mapping for each mutation
mmc3_uniprot_df_select = (
    mmc3_uniprot_df
    .sort_values([
        'uniprot_mutation_pos', 'uniprot_mutation_matches_sequence', 'refseq_mutation_matches_sequence',
         'uniprot_active', 'refseq_active'], ascending=False)
    .drop_duplicates(['refseq_base_id', 'refseq_mutation', 'uniprot_id', 'uniprot_mutation'])
    .sort_values(['uniprot_id', 'uniprot_mutation_pos', 'uniprot_mutation'], ascending=True)
)

In [26]:
df = mmc3_uniprot_df_select
print2("Not 'uniprot_mutation_pos':", (df['uniprot_mutation_pos'] == 0).sum())
print2("Not 'uniprot_mutation_matches_sequence':", (~df['refseq_mutation_matches_sequence']).sum())
print2("Not 'refseq_mutation_matches_sequence':", (~df['refseq_mutation_matches_sequence']).sum())
print2("Not 'uniprot_active':", (df['uniprot_active'] == 'N').sum())
print2("Not 'refseq_active':", (df['refseq_active'] == 'N').sum())
print(df.shape[0], df['index'].drop_duplicates().shape[0])

Not 'uniprot_mutation_pos':                                 6
Not 'uniprot_mutation_matches_sequence':                    13
Not 'refseq_mutation_matches_sequence':                     13
Not 'uniprot_active':                                       0
Not 'refseq_active':                                        13
542 526


In [27]:
# Select valid mappings
mmc3_uniprot_df_good = (
    mmc3_uniprot_df_select[
        (mmc3_uniprot_df_select['uniprot_mutation_pos'] != 0) &
        (df['refseq_mutation_matches_sequence']) &
        (df['refseq_mutation_matches_sequence'])
    ]
)

In [28]:
df = mmc3_uniprot_df_good
print2("Not 'uniprot_mutation_pos':", (df['uniprot_mutation_pos'] == 0).sum())
print2("Not 'uniprot_mutation_matches_sequence':", (~df['refseq_mutation_matches_sequence']).sum())
print2("Not 'refseq_mutation_matches_sequence':", (~df['refseq_mutation_matches_sequence']).sum())
print2("Not 'uniprot_active':", (df['uniprot_active'] == 'N').sum())
print2("Not 'refseq_active':", (df['refseq_active'] == 'N').sum())
print(df.shape[0], df['index'].drop_duplicates().shape[0])

Not 'uniprot_mutation_pos':                                 0
Not 'uniprot_mutation_matches_sequence':                    0
Not 'refseq_mutation_matches_sequence':                     0
Not 'uniprot_active':                                       0
Not 'refseq_active':                                        9
525 522


In [29]:
mmc3_uniprot_df_good.to_pickle(op.join(NOTEBOOK_NAME, 'mmc3_uniprot_df_good.pkl'))

In [30]:
mmc3_uniprot_df_good_bak = mmc3_uniprot_df_good.copy()

### Merge back

In [31]:
mmc3_uniprot_df_good_bak = pd.read_pickle(op.join(NOTEBOOK_NAME, 'mmc3_uniprot_df_good.pkl'))

In [32]:
mmc3_uniprot_df = (
    mmc3_uniprot_df_good_bak
    .drop('index', axis=1)
)

In [33]:
# Merge mmc3 and mmc3_uniprot_df_good
display(HTML("<h4>'mmc3_table1_df3':"))
mmc3_table1_df3 = (
    mmc3_table1_df2.merge(
        mmc3_uniprot_df, 
        on=['refseq_base_id', 'refseq_mutation', 'refseq_mutation_pos'],
        how='left'))
display(mmc3_table1_df3.head(2))
print2(
    "Unique refseq mutations:",
    mmc3_table1_df3.drop_duplicates(['refseq_base_id', 'refseq_mutation']).shape)
print2(
    "Unique uniprot mutations:",
    mmc3_table1_df3.drop_duplicates(['uniprot_id', 'uniprot_mutation']).shape)
print2(
    "Unique refseq interface mutations:",
    mmc3_table1_df3.drop_duplicates(['refseq_base_id', 'refseq_mutation', 'Interactor_Gene_ID']).shape)
print2(
    "Unique uniprot interface mutations:",
    mmc3_table1_df3.drop_duplicates(['uniprot_id', 'uniprot_mutation', 'uniprot_id_2']).shape)


display(HTML("<h4>'mmc3_table2_df3':"))
mmc3_table2_df3 = (
    mmc3_table2_df2.merge(
        mmc3_uniprot_df, 
        on=['refseq_base_id', 'refseq_mutation', 'refseq_mutation_pos'],
        how='left'))
display(mmc3_table2_df3.head(2))
print2(
    "Unique refseq mutations:",
    mmc3_table2_df3.drop_duplicates(['refseq_base_id', 'refseq_mutation']).shape)
print2(
    "Unique uniprot mutations:",
    mmc3_table2_df3.drop_duplicates(['uniprot_id', 'uniprot_mutation']).shape)
print2(
    "Unique refseq interface mutations:",
    mmc3_table2_df3.drop_duplicates(['refseq_base_id', 'refseq_mutation', 'Interactor_Gene_ID']).shape)
print2(
    "Unique uniprot interface mutations:",
    mmc3_table2_df3.drop_duplicates(['uniprot_id', 'uniprot_mutation', 'uniprot_id_2']).shape)


display(HTML("<h4>'mmc3_table3_df3':"))
mmc3_table3_df3 = (
    mmc3_table3_df.merge(
        mmc3_uniprot_df, 
        on=['refseq_base_id', 'refseq_mutation', 'refseq_mutation_pos'],
        how='left'))
display(mmc3_table3_df3.head(2))
print2(
    "Unique refseq mutations:",
    mmc3_table3_df3.drop_duplicates(['refseq_base_id', 'refseq_mutation']).shape)
print2(
    "Unique uniprot mutations:",
    mmc3_table3_df3.drop_duplicates(['uniprot_id', 'uniprot_mutation']).shape)


display(HTML("<h4>All 3 tables combined:"))
print2("Unique mutations:",
    pd.concat(
        [mmc3_table1_df3[['refseq_base_id', 'refseq_mutation']],
         mmc3_table2_df3[['refseq_base_id', 'refseq_mutation']], 
         mmc3_table3_df3[['refseq_base_id', 'refseq_mutation']]],
        ignore_index=True, copy=False)
    .drop_duplicates(['refseq_base_id', 'refseq_mutation']).shape)

,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,Interactor_symbol,Interactor_Gene_ID,Y2H_score,In_Rolland_et_al,refseq_base_id,refseq_mutation,refseq_mutation_pos,refseq_base_id_1,uniprot_id_2,refseq_active,refseq_sequence,uniprot_id,uniprot_mutation_pos,uniprot_active,uniprot_sequence,uniprot_mutation,refseq_mutation_matches_sequence,uniprot_mutation_matches_sequence
0,Wild-type,AANAT,15,15_0,NaN,NaN,BHLHE40,8553,1,1,NaN,NaN,NaN,NP_001079,O14503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wild-type,AANAT,15,15_0,NaN,NaN,BHLHE40,8553,1,1,NaN,NaN,NaN,NP_001079,Q6IB83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Unique refseq mutations:                                    (528, 24)
Unique uniprot mutations:                                   (526, 24)
Unique refseq interface mutations:                          (2004, 24)
Unique uniprot interface mutations:                         (2936, 24)


,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,Interactor_symbol,Interactor_Gene_ID,Interaction_category,GPCA_score1,GPCA_score2,GPCA_average_score,refseq_base_id,refseq_mutation,refseq_mutation_pos,refseq_base_id_1,uniprot_id_2,refseq_active,refseq_sequence,uniprot_id,uniprot_mutation_pos,uniprot_active,uniprot_sequence,uniprot_mutation,refseq_mutation_matches_sequence,uniprot_mutation_matches_sequence
0,Wild-type,AANAT,15,15_0,NaN,NaN,MDFI,4188,Y2H_positive,4.040826,4.698746,4.369786,NaN,NaN,NaN,NP_001079,B1AKB6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wild-type,AANAT,15,15_0,NaN,NaN,MDFI,4188,Y2H_positive,4.040826,4.698746,4.369786,NaN,NaN,NaN,NP_001079,Q99750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Unique refseq mutations:                                    (111, 26)
Unique uniprot mutations:                                   (111, 26)
Unique refseq interface mutations:                          (784, 26)
Unique uniprot interface mutations:                         (1214, 26)


,Category,Symbol,Entrez_Gene_ID,Allele_ID,Mutation_RefSeq_NT,Mutation_RefSeq_AA,Edgotype_class,refseq_base_id,refseq_mutation,refseq_mutation_pos,refseq_active,refseq_sequence,uniprot_id,uniprot_mutation_pos,uniprot_active,uniprot_sequence,uniprot_mutation,refseq_mutation_matches_sequence,uniprot_mutation_matches_sequence
0,Disease mutation,AANAT,15,15_22963,NM_001088:c.385G>A,NP_001079:p.A129T,Edgetic,NP_001079,A129T,129,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,Q16613,129.0,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,A129T,True,True
1,Non-disease variant,AANAT,15,15_900207,NM_001088:c.8C>T,NP_001079:p.T3M,Quasi-wild-type,NP_001079,T3M,3,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,Q16613,3.0,Y,MSTQSTHPLKPEAPRLPPGIPESPSCQRRHTLPASEFRCLTPEDAV...,T3M,True,True


Unique refseq mutations:                                    (527, 19)
Unique uniprot mutations:                                   (526, 19)


Unique mutations:                                           (528, 2)


In [35]:
# Make a copy for future reference
mmc3_table1_df3_bak = mmc3_table1_df3.copy()
mmc3_table2_df3_bak = mmc3_table2_df3.copy()
mmc3_table3_df3_bak = mmc3_table3_df3.copy()

# Combine datasets

### uniprot_interface_mutation_df

In [11]:
# Save a copy for future reference
mmc3_table1_df = pd.read_pickle(op.join(NOTEBOOK_NAME, 'mmc3_table1_df3.pkl'))
mmc3_table2_df = pd.read_pickle(op.join(NOTEBOOK_NAME, 'mmc3_table2_df3.pkl'))
mmc3_table3_df = pd.read_pickle(op.join(NOTEBOOK_NAME, 'mmc3_table3_df3.pkl'))

In [12]:
# Now we need to create an elaspic database with interactions from taipale
mmc3_table1_df.dropna(subset=['uniprot_id', 'uniprot_id_2']).shape == mmc3_table1_df.shape

False

In [13]:
mmc3_table1_df.dropna(subset=['uniprot_id', 'uniprot_id_2']).shape

(2348, 24)

In [14]:
mmc3_table1_df.shape

(3287, 24)

In [15]:
# Uniprot pairs
uniprot_interface_mutation_df = pd.concat([
        mmc3_table1_df[['uniprot_id', 'uniprot_mutation', 'uniprot_id_2']].dropna(),
        mmc3_table2_df[['uniprot_id', 'uniprot_mutation', 'uniprot_id_2']].dropna(),
    ], ignore_index=True).drop_duplicates()

In [16]:
uniprot_interface_mutation_df.shape

(2348, 3)

In [17]:
# Save to database
t = db_remote.import_df(
    uniprot_interface_mutation_df,
    'uniprot_interface_mutation',
    use_temp_file=False,
)

In [18]:
db_remote.create_indexes(
    t.name,
    [(['uniprot_id', 'uniprot_id_2'], False),
     (['uniprot_id_2', 'uniprot_id'], False)]
)